# NEW CODE

In [ ]:
# CRFNet-RS: Semantic Segmentation for Remote Sensing Images with Multiple Sparsity Levels

# Cell 1: Setup Environment
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
from tqdm.notebook import tqdm
from glob import glob
import time
from datetime import datetime
from skimage import io
from scipy import ndimage

# Set random seeds for reproducibility
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Clone the repository
!git clone https://github.com/Ayana-Inria/CRFNet-RS.git
sys.path.append('./CRFNet-RS')

# Install dependencies
!pip install -r ./CRFNet-RS/requirements.txt

In [ ]:
# Cell 2: Fix Issues in Repository
# Fix the unet import issue
with open('./CRFNet-RS/main.py', 'r') as file:
    content = file.read()

# Replace the incorrect import
fixed_content = content.replace('from net.unet import *', '# from net.unet import *')

with open('./CRFNet-RS/main.py', 'w') as file:
    file.write(fixed_content)

# Fix utils_network.py indentation
!sed -i '89s/^/        /' /kaggle/working/CRFNet-RS/utils/utils_network.py

print("Fixed import in main.py and indentation in utils_network.py")

In [ ]:
# Cell 3: Configure Paths and Parameters
# Configuration parameters - these will be the same for all experiments
WINDOW_SIZE = (256, 256)  # Patch size
STRIDE = 32  # Stride for testing inference
IN_CHANNELS = 3  # Number of input channels (RGB/IRRG)
BATCH_SIZE = 10  # Mini-batch size
EPOCHS = 30  # Number of training epochs
SAVE_EPOCH = 10  # Save model every N epochs
BASE_LR = 0.01  # Base learning rate
WEIGHT_DECAY = 0.0005  # Weight decay for optimizer
DATASET_TYPE = "Vaihingen"  # Using Vaihingen dataset

# Default parameters - can be overridden in run_experiment
GT_TYPE = "conncomp"  # Options: "full", "conncomp", "ero"
ERO_DISK_SIZE = 8  # Size of erosion disk for ground truth processing

# Organize folders
DATA_ROOT = "/kaggle/input/potsdamvaihingen/"  # Input data path
OUTPUT_ROOT = "/kaggle/working/"  # Working directory for outputs
WORKING_DATA_ROOT = "/kaggle/working/data"  # Working directory for data processing

# Create necessary directories
os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(WORKING_DATA_ROOT, exist_ok=True)
os.makedirs(f"{WORKING_DATA_ROOT}/top", exist_ok=True)
os.makedirs(f"{WORKING_DATA_ROOT}/gt", exist_ok=True)
os.makedirs(f"{WORKING_DATA_ROOT}/gt_eroded", exist_ok=True)

# Set paths based on dataset
train_ids = ['1', '3', '23', '26', '7', '11', '13', '28', '17', '32', '34', '37']
test_ids = ['5', '15', '21', '30']

# Data file paths
DATA_FILES = f"{DATA_ROOT}/ISPRS_semantic_labeling_Vaihingen/top/top_mosaic_09cm_area{{}}.tif"
LABEL_FILES = f"{DATA_ROOT}/ISPRS_semantic_labeling_Vaihingen/gts_for_participants/top_mosaic_09cm_area{{}}.tif"

# Class labels
LABELS = ["roads", "buildings", "low veg.", "trees", "cars", "clutter"]
N_CLASSES = len(LABELS)

In [ ]:
# Cell 4: Import Required Modules
from dataset.dataset import ISPRS_dataset
from utils.utils_dataset import convert_from_color, convert_to_color, disk, conn_comp
from utils.utils import metrics, sliding_window, count_sliding_window, grouper
from utils.export_result import set_output_location, export_results
from net.net import CRFNet
from net.loss import CrossEntropy2d
from skimage import io
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import cv2
from utils.utils_network import compute_class_weight
from net.test_network import test

print("Imported all required modules")

In [ ]:
# Cell 5: Function to analyze sparsity level of generated ground truth
def analyze_sparsity(gt_type, disk_size, sample_id=None):
    """
    Analyze the sparsity level achieved with a given disk size
    
    Args:
        gt_type: Type of ground truth ("full", "conncomp", "ero")
        disk_size: Size of the erosion disk
        sample_id: Sample ID to analyze (defaults to first training ID)
    
    Returns:
        Percentage of labeled pixels
    """
    if sample_id is None:
        sample_id = train_ids[0]
    
    # Load a sample ground truth image
    sample_gt = io.imread(LABEL_FILES.format(sample_id))
    
    # Convert to label format
    sample_gt_labels = convert_from_color(sample_gt)
    
    # Apply appropriate function based on gt_type
    if gt_type == "full":
        sample_gt_processed = sample_gt_labels
    elif gt_type == "conncomp":
        sample_gt_processed = conn_comp(sample_gt_labels, disk(disk_size))
    else:  # ero
        sample_gt_processed = erode_gt(sample_gt_labels, disk(disk_size))
    
    # Count percentage of labeled pixels
    total_valid_pixels = np.sum(sample_gt_labels != 6)  # Exclude already unlabeled pixels
    labeled_pixels = np.sum((sample_gt_processed != 6) & (sample_gt_labels != 6))
    
    sparse_percentage = (labeled_pixels / total_valid_pixels) * 100
    
    return sparse_percentage, sample_gt_labels, sample_gt_processed

In [ ]:
# Cell 6: Function to find optimal disk size for target sparsity
def find_optimal_disk_size(gt_type, target_percentage, min_disk=1, max_disk=20):
    """
    Find the optimal disk size to achieve a target percentage of labeled pixels
    
    Args:
        gt_type: Type of ground truth ("conncomp", "ero")
        target_percentage: Target percentage of labeled pixels
        min_disk: Minimum disk size to try
        max_disk: Maximum disk size to try
    
    Returns:
        Best disk size
    """
    best_disk_size = min_disk
    best_difference = float('inf')
    
    results = []
    
    for disk_size in range(min_disk, max_disk + 1):
        percentage, _, _ = analyze_sparsity(gt_type, disk_size)
        difference = abs(percentage - target_percentage)
        
        results.append((disk_size, percentage, difference))
        
        if difference < best_difference:
            best_difference = difference
            best_disk_size = disk_size
    
    # Print results table
    print(f"Disk Size | Labeled % | Difference from {target_percentage}%")
    print("-" * 50)
    for disk_size, percentage, difference in results:
        print(f"    {disk_size:<5} | {percentage:7.2f}% | {difference:7.2f}%")
    
    return best_disk_size

In [ ]:
# Cell 7: Define Training Function
def train_model(net, optimizer, scheduler, train_loader, epochs, save_epoch, weights, output_path):
    """
    Train the CRFNet model with proper loss function implementation
    """
    # Initialize loss tracking
    max_iterations = epochs * len(train_loader)
    losses = np.zeros(max_iterations)
    mean_losses = np.zeros(max_iterations)
    iter_ = 0
    
    # Training loop
    for e in tqdm(range(1, epochs + 1), desc="Epochs"):
        # Set model to training mode
        net.train()
        
        # Process each batch
        for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc=f"Epoch {e}", leave=False)):
            # Process targets for multi-scale supervision
            target_np = target.data.cpu().numpy()
            target_np = np.transpose(target_np, [1, 2, 0])
            
            # Create multi-scale targets for different decoder outputs
            target3 = np.transpose(cv2.resize(target_np, dsize=(128, 128), interpolation=cv2.INTER_NEAREST), [2, 0, 1])
            target2 = np.transpose(cv2.resize(target_np, dsize=(64, 64), interpolation=cv2.INTER_NEAREST), [2, 0, 1])
            target1 = np.transpose(cv2.resize(target_np, dsize=(32, 32), interpolation=cv2.INTER_NEAREST), [2, 0, 1])
            target_np = np.transpose(target_np, [2, 0, 1])
            
            # Move data to device
            if torch.cuda.is_available():
                data = Variable(data.cuda())
                target = Variable(torch.from_numpy(target_np).cuda())
                target1_tensor = Variable(torch.from_numpy(target1).type(torch.LongTensor).cuda())
                target2_tensor = Variable(torch.from_numpy(target2).type(torch.LongTensor).cuda())
                target3_tensor = Variable(torch.from_numpy(target3).type(torch.LongTensor).cuda())
            else:
                data = Variable(data)
                target = Variable(torch.from_numpy(target_np))
                target1_tensor = Variable(torch.from_numpy(target1).type(torch.LongTensor))
                target2_tensor = Variable(torch.from_numpy(target2).type(torch.LongTensor))
                target3_tensor = Variable(torch.from_numpy(target3).type(torch.LongTensor))
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass - get all outputs
            output, out_fc, out_neigh, _ = net(data)
            
            # Calculate multi-scale losses as described in the paper
            # 1. Unary loss from main output
            loss = CrossEntropy2d(output, target, weight=weights)
            
            # 2. Multi-scale supervision losses
            if torch.cuda.is_available():
                loss_fc1 = CrossEntropy2d(out_fc[0], target1_tensor, weight=compute_class_weight(target1).cuda())
                loss_fc2 = CrossEntropy2d(out_fc[1], target2_tensor, weight=compute_class_weight(target2).cuda())
                loss_fc3 = CrossEntropy2d(out_fc[2], target3_tensor, weight=compute_class_weight(target3).cuda())
            else:
                loss_fc1 = CrossEntropy2d(out_fc[0], target1_tensor, weight=compute_class_weight(target1))
                loss_fc2 = CrossEntropy2d(out_fc[1], target2_tensor, weight=compute_class_weight(target2))
                loss_fc3 = CrossEntropy2d(out_fc[2], target3_tensor, weight=compute_class_weight(target3))
            
            # 3. Pairwise potential loss from neighborhood layer
            pairwise_loss = CrossEntropy2d(out_neigh, target, weight=weights)
            
            # 4. Combine losses exactly as in the paper
            # Unary loss is average of multi-scale losses plus pairwise loss
            total_loss = (loss + loss_fc1 + loss_fc2 + loss_fc3) / 4 + pairwise_loss
            
            # Backward pass and optimization
            total_loss.backward()
            optimizer.step()
            
            # Record loss
            losses[iter_] = total_loss.item()
            mean_losses[iter_] = np.mean(losses[max(0, iter_-100):iter_+1])
            
            # Display progress every 100 iterations
            if iter_ % 100 == 0:
                # Visualize results if desired
                print(f'Epoch {e}/{epochs} [{batch_idx}/{len(train_loader)} ({100*batch_idx/len(train_loader):.0f}%)] Loss: {total_loss.item():.4f}')
            
            iter_ += 1
        
        # Update learning rate
        scheduler.step()
        
        # Save model checkpoint
        if e % save_epoch == 0:
            model_path = f'{output_path}/model_epoch{e}.pth'
            torch.save(net.state_dict(), model_path)
            print(f"Model saved to {model_path}")
    
    # Save final model
    final_model_path = f'{output_path}/model_final.pth'
    torch.save(net.state_dict(), final_model_path)
    print(f"Final model saved to {final_model_path}")
    return final_model_path

In [ ]:
# Cell 8: Define Testing Function
def test_model(net, test_ids, data_files, label_files, labels, stride, batch_size, window_size, output_path=None):
    """
    Test the model on the provided test data
    """
    # Load test data
    test_images = [1/255 * np.asarray(io.imread(data_files.format(id)), dtype='float32') for id in test_ids]
    test_labels = [np.asarray(io.imread(label_files.format(id)), dtype='uint8') for id in test_ids]
    eroded_labels = [convert_from_color(label) for label in test_labels]
    
    # Run the test
    acc, all_preds, all_gts = test(
        net, test_ids, test_images, test_labels, eroded_labels, 
        labels, stride, batch_size, window_size=window_size, all=True
    )
    
    # Export results
    if output_path:
        title = "Quantitative results for CRFNet testing"
        export_results(
            all_preds, all_gts, 
            os.path.dirname(output_path), os.path.basename(output_path),
            confusionMat=True,
            prodAccuracy=True,
            averageAccuracy=True,
            kappaCoeff=True,
            title=title
        )
        
        # Save prediction images
        for pred, tile_id in zip(all_preds, test_ids):
            img = convert_to_color(pred)
            io.imsave(f"{output_path}/segmentation_result_area{tile_id}.png", img)
    
    return acc, all_preds, all_gts

In [ ]:
# Cell 9: Define experiment runner function for different sparsity levels
def run_experiment(gt_type, gt_percentage=None, disk_size=None):
    """
    Run a complete training and testing cycle for a specific ground truth configuration
    
    Parameters:
    gt_type - Type of ground truth: 'full', 'conncomp', 'ero'
    gt_percentage - For 'conncomp' or 'ero', the percentage of labeled pixels (10 or 30)
    disk_size - Size of erosion disk, if None will be determined automatically
    """
    # Use default disk size if not specified
    if disk_size is None:
        if gt_type == 'full':
            disk_size = ERO_DISK_SIZE  # Default, but not used for full GT
        elif gt_percentage == 30:
            disk_size = 8  # Default size for 30% sparsity (experimentally determined)
        elif gt_percentage == 10:
            disk_size = 12  # Default size for 10% sparsity (experimentally determined)
        else:
            disk_size = ERO_DISK_SIZE
    
    # Create experiment name
    if gt_type == 'full':
        experiment_name = f"CRFNet_{DATASET_TYPE}_full_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    else:
        experiment_name = f"CRFNet_{DATASET_TYPE}_{gt_type}_{gt_percentage}pct_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # Create experiment directory
    os.makedirs(f"{OUTPUT_ROOT}/{experiment_name}", exist_ok=True)
    
    print(f"Starting experiment: {experiment_name}")
    print(f"Dataset: {DATASET_TYPE}")
    print(f"Ground Truth Type: {gt_type}")
    if gt_type != 'full':
        print(f"Target Sparse GT percentage: {gt_percentage}%")
        print(f"Using disk size: {disk_size}")
    
    # Analyze and display actual sparsity level
    if gt_type != 'full':
        actual_percentage, _, _ = analyze_sparsity(gt_type, disk_size)
        print(f"Actual labeled percentage: {actual_percentage:.2f}%")
    
    # Initialize the CRFNet model
    net = CRFNet(n_channels=IN_CHANNELS, n_classes=N_CLASSES, bilinear=True)
    
    # Setup optimizer and learning rate scheduler
    optimizer = optim.SGD(net.parameters(), lr=BASE_LR, momentum=0.9, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [25, 35, 45], gamma=0.1)
    
    # Move model to GPU if available
    if torch.cuda.is_available():
        net.cuda()
        weights = torch.ones(N_CLASSES).cuda()
    else:
        weights = torch.ones(N_CLASSES)
    
    # Create dataset with correct ground truth processing
    train_set = ISPRS_dataset(
        ids=train_ids,
        ids_type='TRAIN',
        gt_type=gt_type,
        gt_modification=disk(disk_size),
        data_files=DATA_FILES,
        label_files=LABEL_FILES,
        window_size=WINDOW_SIZE,
        cache=True,
        augmentation=True
    )
    
    # Create data loader
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE)
    print(f"Created training loader with approximately {len(train_set) // BATCH_SIZE} batches per epoch")
    
    # Train the model
    print("Starting model training...")
    model_path = train_model(
        net=net,
        optimizer=optimizer,
        scheduler=scheduler,
        train_loader=train_loader,
        epochs=EPOCHS,
        save_epoch=SAVE_EPOCH,
        weights=weights,
        output_path=f"{OUTPUT_ROOT}/{experiment_name}"
    )
    print(f"Training completed! Model saved to {model_path}")
    
    # Test the model
    print("Starting model testing...")
    accuracy, all_preds, all_gts = test_model(
        net=net,
        test_ids=test_ids,
        data_files=DATA_FILES,
        label_files=LABEL_FILES,
        labels=LABELS,
        stride=STRIDE,
        batch_size=BATCH_SIZE,
        window_size=WINDOW_SIZE,
        output_path=f"{OUTPUT_ROOT}/{experiment_name}"
    )
    
    print(f"Testing completed with overall accuracy: {accuracy:.2f}%")
    
    # Create a visualization of all test results
    n_images = len(test_ids)
    fig, axes = plt.subplots(n_images, 3, figsize=(15, 5*n_images))
    
    for i, (id, pred, gt) in enumerate(zip(test_ids, all_preds, all_gts)):
        # Load original image
        img = 1/255 * np.asarray(io.imread(DATA_FILES.format(id)), dtype='float32')
        
        # Display original image, ground truth, and prediction
        if n_images > 1:
            axes[i, 0].imshow(np.asarray(255 * img, dtype='uint8'))
            axes[i, 0].set_title(f'Area {id} - Original')
            axes[i, 1].imshow(convert_to_color(gt))
            axes[i, 1].set_title('Ground Truth')
            axes[i, 2].imshow(convert_to_color(pred))
            axes[i, 2].set_title('Prediction')
        else:
            axes[0].imshow(np.asarray(255 * img, dtype='uint8'))
            axes[0].set_title(f'Area {id} - Original')
            axes[1].imshow(convert_to_color(gt))
            axes[1].set_title('Ground Truth')
            axes[2].imshow(convert_to_color(pred))
            axes[2].set_title('Prediction')
    
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_ROOT}/{experiment_name}/all_results.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Results visualization saved to {OUTPUT_ROOT}/{experiment_name}/all_results.png")
    
    # Return experiment results
    return {
        'experiment_name': experiment_name,
        'accuracy': accuracy,
        'model_path': model_path,
        'predictions': all_preds
    }

In [ ]:
# Cell 10: Visualize sparsity levels for different disk sizes
# This cell will help determine the right disk sizes for 10% and 30% sparsity
print("Analyzing sparsity levels for different disk sizes...")
# Try disk sizes from 4 to 16
sparsity_results = []
for disk_size in range(4, 17, 2):
    percentage, _, _ = analyze_sparsity("conncomp", disk_size)
    sparsity_results.append((disk_size, percentage))
    print(f"Disk size {disk_size}: {percentage:.2f}% labeled pixels")

# Plot the results
plt.figure(figsize=(10, 6))
disk_sizes, percentages = zip(*sparsity_results)
plt.plot(disk_sizes, percentages, 'o-', linewidth=2)
plt.axhline(y=30, color='r', linestyle='--', label='30% Target')
plt.axhline(y=10, color='g', linestyle='--', label='10% Target')
plt.xlabel('Disk Size')
plt.ylabel('Percentage of Labeled Pixels')
plt.title('Sparsity Level vs Disk Size')
plt.grid(True)
plt.legend()
plt.show()

# Find optimal disk sizes for 10% and 30% sparsity
disk_size_30pct = find_optimal_disk_size("conncomp", 30)
disk_size_10pct = find_optimal_disk_size("conncomp", 10)

print(f"Recommended disk size for 30% sparsity: {disk_size_30pct}")
print(f"Recommended disk size for 10% sparsity: {disk_size_10pct}")

In [ ]:
# Cell 11: Run all three experiments
results = {}

# 1. Full ground truth experiment
print("=" * 80)
print("RUNNING EXPERIMENT 1/3: FULL GROUND TRUTH")
print("=" * 80)
results['full'] = run_experiment(gt_type='full')

# 2. 30% ground truth experiment
print("\n" + "=" * 80)
print("RUNNING EXPERIMENT 2/3: 30% GROUND TRUTH")
print("=" * 80)
results['30pct'] = run_experiment(gt_type='conncomp', gt_percentage=30, disk_size=disk_size_30pct)

# 3. 10% ground truth experiment
print("\n" + "=" * 80)
print("RUNNING EXPERIMENT 3/3: 10% GROUND TRUTH")
print("=" * 80)
results['10pct'] = run_experiment(gt_type='conncomp', gt_percentage=10, disk_size=disk_size_10pct)

In [ ]:
# Cell 12: Print summary and create comparison chart
print("\n" + "=" * 80)
print("EXPERIMENT RESULTS SUMMARY")
print("=" * 80)
print(f"Full GT: {results['full']['accuracy']:.2f}%")
print(f"30% GT: {results['30pct']['accuracy']:.2f}%")
print(f"10% GT: {results['10pct']['accuracy']:.2f}%")
print("=" * 80)

# Create a bar chart comparing the results
plt.figure(figsize=(10, 6))
labels = ['Full GT', '30% GT', '10% GT']
values = [results['full']['accuracy'], results['30pct']['accuracy'], results['10pct']['accuracy']]
plt.bar(labels, values, color=['green', 'blue', 'orange'])
plt.axhline(y=85, color='r', linestyle='--', label='Paper 30% Result')
plt.axhline(y=81, color='purple', linestyle='--', label='Paper 10% Result')
plt.axhline(y=90, color='green', linestyle='--', label='Paper Full Result')
plt.ylabel('Overall Accuracy (%)')
plt.title('CRFNet Performance on Vaihingen Dataset')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.savefig(f"{OUTPUT_ROOT}/accuracy_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print("All experiments completed successfully!")